# Locals that your facebook friends visit more

- 

## Install_the_librarys

In [179]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install statistics

### API >> https://facebook-sdk.readthedocs.io/en/latest/api.html
### Get access token >> https://developers.facebook.com/tools/explorer

## Creating_the_fecebook_graph_API_access

In [180]:
#imports
import facebook
import pandas as pd
import numpy as np
from tqdm import tqdm
import folium
from folium.plugins import HeatMap

token = 'EAACEdEose0cBAKsOuHScVFjhAs1Amq6L4zHxejnaIwcwHSIUu8xmEdVxIwN2yNBgtqgi6LaghWtHkD41NRJn96B8VPWgkMuFVx8tvd8DHIsc8ZBxwFVMWh8GhuD825ThC0nO60bIJtZBmHqh0V3hGPfgl3dRMYJxcZC5TIWAe0bRQAD0Lo4Ew80w0A1BLz11b9xkZC0OhQZDZD' 
graph = facebook.GraphAPI(access_token=token, version='2.2')

## Getting_the_friends

In [181]:
##Pegar os lugares dos amigos e os meus
friends = graph.get_object("me/friends") #Amigos
me_places = graph.get_object("me/tagged_places")#Meus
#friends['summary']['total_count']

friendsStartList = friends['data']
my_placesList = me_places["data"]

## Get_each_user_locations

In [183]:
locations = []

#

for friendDict in range(0, len(friendsStartList)):
    user_id = friendsStartList[friendDict]['id']
    user_name = friendsStartList[friendDict]['name']
    locations.append({'id': user_id, 'name': user_name, 'locations': []})
    friend_tagged = graph.get_object(str(user_id) + "/tagged_places")
    for place in friend_tagged['data']:
        locations[friendDict]['locations'].append(place)

#Limpando o Location de dados vazios
for friendDict in range(len(locations)-1, -1, -1):
    if len(locations[friendDict]['locations']) == 0:
        del locations[friendDict]

In [184]:
#Criando Manualmente um DataFrame

df = pd.DataFrame(columns=('ID', 'User_name', 'Place_Name', "LAT", "LON"))
df

,ID,User_name,Place_Name,LAT,LON


In [185]:
#Inserindo os dados dos amigos
for friendDict in range(0, len(locations)):
    #print(locations[friendDict]["id"],locations[friendDict]["name"] )
    for atual in locations[friendDict]['locations']:
        #print(atual["place"], "\n")
        df.loc[len(df)+1] = [locations[friendDict]["id"], locations[friendDict]["name"], atual["place"]["name"],
                      atual["place"]["location"]["latitude"], atual["place"]["location"]["longitude"]]
        
#Inserindo meus dados
for friendDict in range(0, len(my_placesList)):
    #print(my_placesList[friendDict]["id"], "Me", my_placesList[friendDict]["place"]["name"])
    df.loc[len(df)+1] = [my_placesList[friendDict]["id"],"Me", my_placesList[friendDict]["place"]["name"],
                      my_placesList[friendDict]["place"]["location"]["latitude"], my_placesList[friendDict]["place"]["location"]["longitude"]]


In [186]:
#Análise do resultado
df.head()

,ID,User_name,Place_Name,LAT,LON
1,10204215934936399,Luan Tavares,"Salto do Itiquira, Formosa, Goiás",-15.792714,-47.893775
2,10204215934936399,Luan Tavares,Reitoria UFRN,-5.839705,-35.201467
3,10204215934936399,Luan Tavares,Whiskritorio Natal,-5.863116,-35.193069
4,786112898127224,Nalbert Gabriel,Imd- Ufrn,-5.832736,-35.205368
5,786112898127224,Nalbert Gabriel,Boqueirão,-7.483330,-36.116700


In [187]:
#Criando lista de lugares e contando a reccorrencia
locais_unique = df["Place_Name"].unique().tolist()

dataFinal = df.copy()
dataFinal["Count"] = 0

dataFinal = pd.DataFrame(dataFinal.groupby(["Place_Name", "LAT", "LON"])['Count'].count()).reset_index()

#Lista de lugares visitados em ordem decrescente
dataFinal.sort_values("Count", inplace=True, ascending=False)
dataFinal.head()

,Place_Name,LAT,LON,Count
5,Chapada Diamantina,-12.561809,-41.390053,2
6,Chiquinho Sorvetes - Natal,-5.851693,-35.203580,2
18,Morro do Pai Inácio,-12.459120,-41.471930,2
36,Restaurante Ponto Certo,-6.461359,-37.097130,2
9,Farol da Barra,-13.008230,-38.530095,2


In [188]:
###HeatMap
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

coordinates = []

for i in range(len(dataFinal)):
    # eliminate items with'nan' element
    if all(~np.isnan([dataFinal.ix[i,'LAT'], dataFinal.ix[i,'LON'], dataFinal.ix[i,'Count']])):
        coordinates.append([dataFinal.ix[i,'LAT'], dataFinal.ix[i,'LON'], dataFinal.ix[i,'Count']])

# Create map object
htMap = folium.Map(location=mapc, zoom_start=zoom)

HeatMap(coordinates).add_to(htMap)

htMap

In [189]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Create map object
map_osm = folium.Map(location=mapc, zoom_start=zoom)

dataFinal = dataFinal.reset_index()

# Plot each of the locations that we geocoded
for j in tqdm(range(len(dataFinal))):
    if (j < 5):
        folium.Marker([dataFinal.ix[j,'LAT'], dataFinal.ix[j,'LON']],
                      popup=(str(j+1)+"º - "+dataFinal.ix[j,'Place_Name']),
                      icon = folium.Icon(color ='red')).add_to(map_osm)
    else:
        folium.Marker([dataFinal.ix[j,'LAT'], dataFinal.ix[j,'LON']],
                      popup=(str(j+1)+"º - "+dataFinal.ix[j,'Place_Name'])).add_to(map_osm)
# Show the map
map_osm

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 43.16it/s]
